#### Imports & config

In [2]:
# Cell 1: Imports and display options
import os
import gc
import numpy as np
import pandas as pd

# display
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 180)


In [8]:
# Cell 2: File paths - edit if needed
DATA_DIR = r"C:\Home Credit Deafault Risk\data\home_credit"   # adjust relative path to your notebook location
OUT_DIR = "../outputs"
os.makedirs(OUT_DIR, exist_ok=True)

FILES = {
    "app": os.path.join(DATA_DIR, "application_train.csv"),
    "bureau": os.path.join(DATA_DIR, "bureau.csv"),
    "prev": os.path.join(DATA_DIR, "previous_application.csv"),
    "inst": os.path.join(DATA_DIR, "installments_payments.csv"),
    "cc_bal": os.path.join(DATA_DIR, "credit_card_balance.csv"),
}


In [20]:
# Cell 3: Memory reduction function (Modified to handle Categories)
def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        
        # FIX: Check if the column is NOT 'object' AND NOT 'category'
        # We only want to check min/max for true numeric types.
        if col_type != object and str(col_type) != 'category': 
            
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type).startswith('int'):
                # Integer downcasting logic
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                # Float downcasting logic
                df[col] = pd.to_numeric(df[col], downcast='float')
        
        # Original logic for 'object' types (converting to category) remains
        elif col_type == object:
            df[col] = df[col].astype('category')
            
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose:
        print(f'Memory usage: {start_mem:.2f} MB -> {end_mem:.2f} MB ({100*(start_mem-end_mem)/start_mem:.1f}% reduction)')
    return df


In [21]:
# Cell 4: Load main application_train
app = pd.read_csv(FILES['app'])
print("application_train shape:", app.shape)
app = reduce_mem_usage(app)
app.head(3)


application_train shape: (307511, 122)
Memory usage: 504.99 MB -> 96.50 MB (80.9% reduction)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [22]:
# Cell 5: Load other tables
bureau = pd.read_csv(FILES['bureau'])
prev = pd.read_csv(FILES['prev'])
inst = pd.read_csv(FILES['inst'])
cc_bal = pd.read_csv(FILES['cc_bal'])

print("bureau:", bureau.shape, "previous_application:", prev.shape,
      "installments_payments:", inst.shape, "credit_card_balance:", cc_bal.shape)

# Reduce memory
bureau = reduce_mem_usage(bureau)
prev = reduce_mem_usage(prev)
inst = reduce_mem_usage(inst)
cc_bal = reduce_mem_usage(cc_bal)


bureau: (1716428, 17) previous_application: (1670214, 37) installments_payments: (13605401, 8) credit_card_balance: (3840312, 23)
Memory usage: 472.82 MB -> 124.41 MB (73.7% reduction)
Memory usage: 1703.01 MB -> 178.41 MB (89.5% reduction)
Memory usage: 830.41 MB -> 493.05 MB (40.6% reduction)
Memory usage: 846.39 MB -> 454.14 MB (46.3% reduction)


In [23]:
# Cell 6: Sanity checks
print("application columns:", app.columns.tolist()[:10])
print("bureau columns sample:", bureau.columns.tolist()[:10])
print("previous_application sample columns:", prev.columns.tolist()[:10])
print("installments_payments columns:", inst.columns.tolist()[:10])
print("credit_card_balance columns:", cc_bal.columns.tolist()[:10])

# Key existence
print("Is SK_ID_CURR in app?", "SK_ID_CURR" in app.columns)
print("Is SK_ID_CURR in bureau?", "SK_ID_CURR" in bureau.columns)
print("Is SK_ID_CURR in previous_application?", "SK_ID_CURR" in prev.columns)
print("Is SK_ID_PREV in installments_payments?", "SK_ID_PREV" in inst.columns)
print("Is SK_ID_PREV in credit_card_balance?", "SK_ID_PREV" in cc_bal.columns)


application columns: ['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY']
bureau columns sample: ['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG']
previous_application sample columns: ['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START']
installments_payments columns: ['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION', 'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT', 'AMT_INSTALMENT', 'AMT_PAYMENT']
credit_card_balance columns: ['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT', 'AMT_DRAWIN

In [24]:
# Cell 7: Aggregation utility
def agg_numeric(df, group_var, df_name):
    # Numeric aggregations
    numeric_df = df.select_dtypes(include=[np.number])
    numeric_df[group_var] = df[group_var]
    agg = numeric_df.groupby(group_var).agg(['count','mean','max','min','sum','std']).reset_index()
    # flatten column names
    agg.columns = [group_var] + [f"{df_name}__{c[0]}__{c[1]}" for c in agg.columns.tolist()[1:]]
    return agg

def agg_categorical(df, group_var, df_name):
    categorical = df.select_dtypes(include=['category','object'])
    categorical[group_var] = df[group_var]
    # one-hot and aggregate
    cat_ohe = pd.get_dummies(categorical.drop(columns=[group_var]))
    cat_ohe[group_var] = categorical[group_var].values
    agg = cat_ohe.groupby(group_var).agg(['sum','mean']).reset_index()
    agg.columns = [group_var] + [f"{df_name}__{c}__{agg_level}" 
                                 for c, agg_level in [(col, ag) for col in cat_ohe.columns if col != group_var for ag in ['sum','mean']]]
    # note: above is a generic name - but can be big; simpler approach below for main datasets
    return agg


In [25]:
# Cell 8: bureau aggregates (per SK_ID_CURR)
# select numeric cols to aggregate
num_cols_bureau = [c for c in bureau.columns if bureau[c].dtype in [np.float64, np.int64, np.int32, np.float32, np.int16, np.int8] and c not in ['SK_ID_BUREAU', 'SK_ID_CURR']]
print("bureau numeric columns to agg:", num_cols_bureau[:20])

bureau_agg = bureau.groupby('SK_ID_CURR')[num_cols_bureau].agg(['count','mean','max','min','sum','std']).reset_index()
# flatten names
bureau_agg.columns = ['SK_ID_CURR'] + [f"bureau__{c}__{stat}" for c, stat in bureau_agg.columns.tolist()[1:]]
print("bureau_agg shape:", bureau_agg.shape)
bureau_agg.head(3)


bureau numeric columns to agg: ['DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY']
bureau_agg shape: (305811, 73)


,SK_ID_CURR,bureau__DAYS_CREDIT__count,bureau__DAYS_CREDIT__mean,bureau__DAYS_CREDIT__max,bureau__DAYS_CREDIT__min,bureau__DAYS_CREDIT__sum,bureau__DAYS_CREDIT__std,bureau__CREDIT_DAY_OVERDUE__count,bureau__CREDIT_DAY_OVERDUE__mean,bureau__CREDIT_DAY_OVERDUE__max,bureau__CREDIT_DAY_OVERDUE__min,bureau__CREDIT_DAY_OVERDUE__sum,bureau__CREDIT_DAY_OVERDUE__std,bureau__DAYS_CREDIT_ENDDATE__count,bureau__DAYS_CREDIT_ENDDATE__mean,bureau__DAYS_CREDIT_ENDDATE__max,bureau__DAYS_CREDIT_ENDDATE__min,bureau__DAYS_CREDIT_ENDDATE__sum,bureau__DAYS_CREDIT_ENDDATE__std,bureau__DAYS_ENDDATE_FACT__count,bureau__DAYS_ENDDATE_FACT__mean,bureau__DAYS_ENDDATE_FACT__max,bureau__DAYS_ENDDATE_FACT__min,bureau__DAYS_ENDDATE_FACT__sum,bureau__DAYS_ENDDATE_FACT__std,bureau__AMT_CREDIT_MAX_OVERDUE__count,bureau__AMT_CREDIT_MAX_OVERDUE__mean,bureau__AMT_CREDIT_MAX_OVERDUE__max,bureau__AMT_CREDIT_MAX_OVERDUE__min,bureau__AMT_CREDIT_MAX_OVERDUE__sum,bureau__AMT_CREDIT_MAX_OVERDUE__std,bureau__CNT_CREDIT_PROLONG__count,bureau__CNT_CREDIT_PROLONG__mean,bureau__CNT_CREDIT_PROLONG__max,bureau__CNT_CREDIT_PROLONG__min,bureau__CNT_CREDIT_PROLONG__sum,bureau__CNT_CREDIT_PROLONG__std,bureau__AMT_CREDIT_SUM__count,bureau__AMT_CREDIT_SUM__mean,bureau__AMT_CREDIT_SUM__max,bureau__AMT_CREDIT_SUM__min,bureau__AMT_CREDIT_SUM__sum,bureau__AMT_CREDIT_SUM__std,bureau__AMT_CREDIT_SUM_DEBT__count,bureau__AMT_CREDIT_SUM_DEBT__mean,bureau__AMT_CREDIT_SUM_DEBT__max,bureau__AMT_CREDIT_SUM_DEBT__min,bureau__AMT_CREDIT_SUM_DEBT__sum,bureau__AMT_CREDIT_SUM_DEBT__std,bureau__AMT_CREDIT_SUM_LIMIT__count,bureau__AMT_CREDIT_SUM_LIMIT__mean,bureau__AMT_CREDIT_SUM_LIMIT__max,bureau__AMT_CREDIT_SUM_LIMIT__min,bureau__AMT_CREDIT_SUM_LIMIT__sum,bureau__AMT_CREDIT_SUM_LIMIT__std,bureau__AMT_CREDIT_SUM_OVERDUE__count,bureau__AMT_CREDIT_SUM_OVERDUE__mean,bureau__AMT_CREDIT_SUM_OVERDUE__max,bureau__AMT_CREDIT_SUM_OVERDUE__min,bureau__AMT_CREDIT_SUM_OVERDUE__sum,bureau__AMT_CREDIT_SUM_OVERDUE__std,bureau__DAYS_CREDIT_UPDATE__count,bureau__DAYS_CREDIT_UPDATE__mean,bureau__DAYS_CREDIT_UPDATE__max,bureau__DAYS_CREDIT_UPDATE__min,bureau__DAYS_CREDIT_UPDATE__sum,bureau__DAYS_CREDIT_UPDATE__std,bureau__AMT_ANNUITY__count,bureau__AMT_ANNUITY__mean,bureau__AMT_ANNUITY__max,bureau__AMT_ANNUITY__min,bureau__AMT_ANNUITY__sum,bureau__AMT_ANNUITY__std
0,100001,7,-735.00,-49,-1572,-5145,489.942514,7,0.0,0,0,0,0.0,7,82.428574,1778.0,-1329.0,577.0,1032.859277,4,-825.500000,-544.0,-1328.0,-3302.0,369.078583,0,NaN,NaN,NaN,0.000,NaN,7,0.0,0,0,0,0.0,7,207623.571429,378000.0,85500.0,1453365.000,122544.544510,7,85240.928571,373239.0,0.0,596686.5,137485.631124,6,0.00000,0.000,0.0,0.000,0.0000,7,0.0,0.0,0.0,0.0,0.0,7,-93.142857,-6,-155,-652,77.204120,7,3545.357143,10822.5,0.0,24817.5,4800.607529
1,100002,8,-874.00,-103,-1437,-6992,431.451040,8,0.0,0,0,0,0.0,6,-349.000000,780.0,-1072.0,-2094.0,767.490977,6,-697.500000,-36.0,-1185.0,-4185.0,515.992554,5,1681.029,5043.645,0.0,8405.145,2363.2469,8,0.0,0,0,0,0.0,8,108131.945625,450000.0,0.0,865055.565,146075.557435,5,49156.200000,245781.0,0.0,245781.0,109916.604716,4,7997.14125,31988.565,0.0,31988.565,15994.2825,8,0.0,0.0,0.0,0.0,0.0,8,-499.875000,-7,-1185,-3999,518.522472,7,0.000000,0.0,0.0,0.0,0.000000
2,100003,4,-1400.75,-606,-2586,-5603,909.826128,4,0.0,0,0,0,0.0,4,-544.500000,1216.0,-2434.0,-2178.0,1492.770467,3,-1097.333374,-540.0,-2131.0,-3292.0,896.097290,4,0.000,0.000,0.0,0.000,0.0000,4,0.0,0,0,0,0.0,4,254350.125000,810000.0,22248.0,1017400.500,372269.465535,4,0.000000,0.0,0.0,0.0,0.000000,4,202500.00000,810000.000,0.0,810000.000,405000.0000,4,0.0,0.0,0.0,0.0,0.0,4,-816.000000,-43,-2131,-3264,908.053963,0,NaN,NaN,NaN,0.0,NaN


In [26]:
# Cell 9: previous_application aggregates
num_cols_prev = [c for c in prev.columns if prev[c].dtype in [np.float64, np.int64, np.int32, np.float32, np.int16, np.int8] and c not in ['SK_ID_PREV','SK_ID_CURR']]
print("previous_application numeric to agg:", num_cols_prev[:20])

prev_agg = prev.groupby('SK_ID_CURR')[num_cols_prev].agg(['count','mean','max','min','sum','std']).reset_index()
prev_agg.columns = ['SK_ID_CURR'] + [f"prev__{c}__{stat}" for c, stat in prev_agg.columns.tolist()[1:]]
print("prev_agg shape:", prev_agg.shape)
prev_agg.head(3)


previous_application numeric to agg: ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE', 'HOUR_APPR_PROCESS_START', 'NFLAG_LAST_APPL_IN_DAY', 'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY', 'RATE_INTEREST_PRIVILEGED', 'DAYS_DECISION', 'SELLERPLACE_AREA', 'CNT_PAYMENT', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL']
prev_agg shape: (338857, 115)


,SK_ID_CURR,prev__AMT_ANNUITY__count,prev__AMT_ANNUITY__mean,prev__AMT_ANNUITY__max,prev__AMT_ANNUITY__min,prev__AMT_ANNUITY__sum,prev__AMT_ANNUITY__std,prev__AMT_APPLICATION__count,prev__AMT_APPLICATION__mean,prev__AMT_APPLICATION__max,prev__AMT_APPLICATION__min,prev__AMT_APPLICATION__sum,prev__AMT_APPLICATION__std,prev__AMT_CREDIT__count,prev__AMT_CREDIT__mean,prev__AMT_CREDIT__max,prev__AMT_CREDIT__min,prev__AMT_CREDIT__sum,prev__AMT_CREDIT__std,prev__AMT_DOWN_PAYMENT__count,prev__AMT_DOWN_PAYMENT__mean,prev__AMT_DOWN_PAYMENT__max,prev__AMT_DOWN_PAYMENT__min,prev__AMT_DOWN_PAYMENT__sum,prev__AMT_DOWN_PAYMENT__std,prev__AMT_GOODS_PRICE__count,prev__AMT_GOODS_PRICE__mean,prev__AMT_GOODS_PRICE__max,prev__AMT_GOODS_PRICE__min,prev__AMT_GOODS_PRICE__sum,prev__AMT_GOODS_PRICE__std,prev__HOUR_APPR_PROCESS_START__count,prev__HOUR_APPR_PROCESS_START__mean,prev__HOUR_APPR_PROCESS_START__max,prev__HOUR_APPR_PROCESS_START__min,prev__HOUR_APPR_PROCESS_START__sum,prev__HOUR_APPR_PROCESS_START__std,prev__NFLAG_LAST_APPL_IN_DAY__count,prev__NFLAG_LAST_APPL_IN_DAY__mean,prev__NFLAG_LAST_APPL_IN_DAY__max,prev__NFLAG_LAST_APPL_IN_DAY__min,prev__NFLAG_LAST_APPL_IN_DAY__sum,prev__NFLAG_LAST_APPL_IN_DAY__std,prev__RATE_DOWN_PAYMENT__count,prev__RATE_DOWN_PAYMENT__mean,prev__RATE_DOWN_PAYMENT__max,prev__RATE_DOWN_PAYMENT__min,prev__RATE_DOWN_PAYMENT__sum,prev__RATE_DOWN_PAYMENT__std,prev__RATE_INTEREST_PRIMARY__count,prev__RATE_INTEREST_PRIMARY__mean,prev__RATE_INTEREST_PRIMARY__max,prev__RATE_INTEREST_PRIMARY__min,prev__RATE_INTEREST_PRIMARY__sum,prev__RATE_INTEREST_PRIMARY__std,prev__RATE_INTEREST_PRIVILEGED__count,prev__RATE_INTEREST_PRIVILEGED__mean,prev__RATE_INTEREST_PRIVILEGED__max,prev__RATE_INTEREST_PRIVILEGED__min,prev__RATE_INTEREST_PRIVILEGED__sum,prev__RATE_INTEREST_PRIVILEGED__std,prev__DAYS_DECISION__count,prev__DAYS_DECISION__mean,prev__DAYS_DECISION__max,prev__DAYS_DECISION__min,prev__DAYS_DECISION__sum,prev__DAYS_DECISION__std,prev__SELLERPLACE_AREA__count,prev__SELLERPLACE_AREA__mean,prev__SELLERPLACE_AREA__max,prev__SELLERPLACE_AREA__min,prev__SELLERPLACE_AREA__sum,prev__SELLERPLACE_AREA__std,prev__CNT_PAYMENT__count,prev__CNT_PAYMENT__mean,prev__CNT_PAYMENT__max,prev__CNT_PAYMENT__min,prev__CNT_PAYMENT__sum,prev__CNT_PAYMENT__std,prev__DAYS_FIRST_DRAWING__count,prev__DAYS_FIRST_DRAWING__mean,prev__DAYS_FIRST_DRAWING__max,prev__DAYS_FIRST_DRAWING__min,prev__DAYS_FIRST_DRAWING__sum,prev__DAYS_FIRST_DRAWING__std,prev__DAYS_FIRST_DUE__count,prev__DAYS_FIRST_DUE__mean,prev__DAYS_FIRST_DUE__max,prev__DAYS_FIRST_DUE__min,prev__DAYS_FIRST_DUE__sum,prev__DAYS_FIRST_DUE__std,prev__DAYS_LAST_DUE_1ST_VERSION__count,prev__DAYS_LAST_DUE_1ST_VERSION__mean,prev__DAYS_LAST_DUE_1ST_VERSION__max,prev__DAYS_LAST_DUE_1ST_VERSION__min,prev__DAYS_LAST_DUE_1ST_VERSION__sum,prev__DAYS_LAST_DUE_1ST_VERSION__std,prev__DAYS_LAST_DUE__count,prev__DAYS_LAST_DUE__mean,prev__DAYS_LAST_DUE__max,prev__DAYS_LAST_DUE__min,prev__DAYS_LAST_DUE__sum,prev__DAYS_LAST_DUE__std,prev__DAYS_TERMINATION__count,prev__DAYS_TERMINATION__mean,prev__DAYS_TERMINATION__max,prev__DAYS_TERMINATION__min,prev__DAYS_TERMINATION__sum,prev__DAYS_TERMINATION__std,prev__NFLAG_INSURED_ON_APPROVAL__count,prev__NFLAG_INSURED_ON_APPROVAL__mean,prev__NFLAG_INSURED_ON_APPROVAL__max,prev__NFLAG_INSURED_ON_APPROVAL__min,prev__NFLAG_INSURED_ON_APPROVAL__sum,prev__NFLAG_INSURED_ON_APPROVAL__std
0,100001,1,3951.000,3951.000,3951.000,3951.000,NaN,1,24835.5,24835.5,24835.5,24835.5,NaN,1,23787.0,23787.0,23787.0,23787.0,NaN,1,2520.0,2520.0,2520.0,2520.0,NaN,1,24835.5,24835.5,24835.5,24835.5,NaN,1,13.000000,13,13,13,NaN,1,1.0,1,1,1,NaN,1,0.104326,0.104326,0.104326,0.104326,NaN,0,NaN,NaN,NaN,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,1,-1740.0,-1740,-1740,-1740,NaN,1,23.0,23,23,23,NaN,1,8.0,8.0,8.0,8.0,NaN,1,365243.0,365243.0,365243.0,365243.0,NaN,1,-1709.000000,-1709.0,-1709.0,-1709.0,NaN,1,-1499.000000,-1499.0,-1499.0,-1499.0,NaN,1,-1619.000000,-1619.0,-1619.0,-1619.0,NaN,1,-1612.000000,-1612.0,-1612.0,-1612.0,NaN

In [27]:
# Cell 10: installments payments agg
# common useful features: AMT_PAYMENT, DAYS_ENTRY_PAYMENT, AMT_INSTALMENT
cols_inst = [c for c in inst.columns if c in ['AMT_PAYMENT','DAYS_ENTRY_PAYMENT','AMT_INSTALMENT','NUM_INSTALMENT_VERSION']]
cols_inst = [c for c in cols_inst if c in inst.columns]
print("inst cols:", cols_inst)

inst_agg = inst.groupby('SK_ID_CURR')[cols_inst].agg(['count','mean','max','min','sum','std']).reset_index()
inst_agg.columns = ['SK_ID_CURR'] + [f"inst__{c}__{stat}" for c, stat in inst_agg.columns.tolist()[1:]]
print("inst_agg shape:", inst_agg.shape)
inst_agg.head(3)


inst cols: ['NUM_INSTALMENT_VERSION', 'DAYS_ENTRY_PAYMENT', 'AMT_INSTALMENT', 'AMT_PAYMENT']
inst_agg shape: (339587, 25)


,SK_ID_CURR,inst__NUM_INSTALMENT_VERSION__count,inst__NUM_INSTALMENT_VERSION__mean,inst__NUM_INSTALMENT_VERSION__max,inst__NUM_INSTALMENT_VERSION__min,inst__NUM_INSTALMENT_VERSION__sum,inst__NUM_INSTALMENT_VERSION__std,inst__DAYS_ENTRY_PAYMENT__count,inst__DAYS_ENTRY_PAYMENT__mean,inst__DAYS_ENTRY_PAYMENT__max,inst__DAYS_ENTRY_PAYMENT__min,inst__DAYS_ENTRY_PAYMENT__sum,inst__DAYS_ENTRY_PAYMENT__std,inst__AMT_INSTALMENT__count,inst__AMT_INSTALMENT__mean,inst__AMT_INSTALMENT__max,inst__AMT_INSTALMENT__min,inst__AMT_INSTALMENT__sum,inst__AMT_INSTALMENT__std,inst__AMT_PAYMENT__count,inst__AMT_PAYMENT__mean,inst__AMT_PAYMENT__max,inst__AMT_PAYMENT__min,inst__AMT_PAYMENT__sum,inst__AMT_PAYMENT__std
0,100001,7,1.142857,2.0,1.0,8.0,0.377964,7,-2195.000000,-1628.0,-2916.0,-15365.0,643.904236,7,5885.132143,17397.900,3951.000,41195.925,5076.676624,7,5885.132143,17397.900,3951.000,41195.925,5076.676624
1,100002,19,1.052632,2.0,1.0,20.0,0.229416,19,-315.421051,-49.0,-587.0,-5993.0,172.058884,19,11559.247105,53093.745,9251.775,219625.695,10058.037722,19,11559.247105,53093.745,9251.775,219625.695,10058.037722
2,100003,25,1.040000,2.0,1.0,26.0,0.200000,25,-1385.319946,-544.0,-2324.0,-34633.0,757.325439,25,64754.586000,560835.360,6662.970,1618864.650,110542.592300,25,64754.586000,560835.360,6662.970,1618864.650,110542.592300


In [28]:
# Cell 11: credit card balance aggregations
# choose representative columns: AMT_BALANCE, AMT_CREDIT_LIMIT_ACTUAL, AMT_DRAWINGS_CURRENT, AMT_PAYMENT_CURRENT
cols_cc = [c for c in cc_bal.columns if c in ['AMT_BALANCE','AMT_CREDIT_LIMIT_ACTUAL','AMT_DRAWINGS_CURRENT','AMT_PAYMENT_CURRENT','SK_DPD','SK_DPD_DEF']]
cols_cc = [c for c in cols_cc if c in cc_bal.columns]
print("credit card cols:", cols_cc)

cc_agg = cc_bal.groupby('SK_ID_CURR')[cols_cc].agg(['count','mean','max','min','sum','std']).reset_index()
cc_agg.columns = ['SK_ID_CURR'] + [f"cc__{c}__{stat}" for c, stat in cc_agg.columns.tolist()[1:]]
print("cc_agg shape:", cc_agg.shape)
cc_agg.head(3)


credit card cols: ['AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_CURRENT', 'AMT_PAYMENT_CURRENT', 'SK_DPD', 'SK_DPD_DEF']
cc_agg shape: (103558, 37)


,SK_ID_CURR,cc__AMT_BALANCE__count,cc__AMT_BALANCE__mean,cc__AMT_BALANCE__max,cc__AMT_BALANCE__min,cc__AMT_BALANCE__sum,cc__AMT_BALANCE__std,cc__AMT_CREDIT_LIMIT_ACTUAL__count,cc__AMT_CREDIT_LIMIT_ACTUAL__mean,cc__AMT_CREDIT_LIMIT_ACTUAL__max,cc__AMT_CREDIT_LIMIT_ACTUAL__min,cc__AMT_CREDIT_LIMIT_ACTUAL__sum,cc__AMT_CREDIT_LIMIT_ACTUAL__std,cc__AMT_DRAWINGS_CURRENT__count,cc__AMT_DRAWINGS_CURRENT__mean,cc__AMT_DRAWINGS_CURRENT__max,cc__AMT_DRAWINGS_CURRENT__min,cc__AMT_DRAWINGS_CURRENT__sum,cc__AMT_DRAWINGS_CURRENT__std,cc__AMT_PAYMENT_CURRENT__count,cc__AMT_PAYMENT_CURRENT__mean,cc__AMT_PAYMENT_CURRENT__max,cc__AMT_PAYMENT_CURRENT__min,cc__AMT_PAYMENT_CURRENT__sum,cc__AMT_PAYMENT_CURRENT__std,cc__SK_DPD__count,cc__SK_DPD__mean,cc__SK_DPD__max,cc__SK_DPD__min,cc__SK_DPD__sum,cc__SK_DPD__std,cc__SK_DPD_DEF__count,cc__SK_DPD_DEF__mean,cc__SK_DPD_DEF__max,cc__SK_DPD_DEF__min,cc__SK_DPD_DEF__sum,cc__SK_DPD_DEF__std
0,100006,6,0.000000,0.00,0.0,0.000,0.000000,6,270000.000000,270000,270000,1620000,0.000000,6,0.000000,0.0,0.0,0.0,0.000000,0,NaN,NaN,NaN,0.00,NaN,6,0.000000,0,0,0,0.000000,6,0.000000,0,0,0,0.000000
1,100011,74,54482.111149,189000.00,0.0,4031676.225,68127.238270,74,164189.189189,180000,90000,12150000,34482.743620,74,2432.432432,180000.0,0.0,180000.0,20924.574974,74,4843.064189,55485.0,0.0,358386.75,7279.601955,74,0.000000,0,0,0,0.000000,74,0.000000,0,0,0,0.000000
2,100013,96,18159.919219,161420.22,0.0,1743352.245,43237.406997,96,131718.750000,157500,45000,12645000,47531.585759,96,5953.125000,157500.0,0.0,571500.0,27843.366225,96,7168.346250,153675.0,0.0,688161.24,21626.144325,96,0.010417,1,0,1,0.102062,96,0.010417,1,0,1,0.102062


In [29]:
# Cell 12: Merge aggregates into main application dataframe
print("app before merge:", app.shape)
df = app.merge(bureau_agg, on='SK_ID_CURR', how='left')
print("after bureau:", df.shape)
df = df.merge(prev_agg, on='SK_ID_CURR', how='left')
print("after prev:", df.shape)
df = df.merge(inst_agg, on='SK_ID_CURR', how='left')
print("after inst:", df.shape)
df = df.merge(cc_agg, on='SK_ID_CURR', how='left')
print("after cc:", df.shape)

# free memory
del bureau_agg, prev_agg, inst_agg, cc_agg, bureau, prev, inst, cc_bal
gc.collect()

df = reduce_mem_usage(df)
df.head(3)


app before merge: (307511, 122)
after bureau: (307511, 194)
after prev: (307511, 308)
after inst: (307511, 332)
after cc: (307511, 368)
Memory usage: 598.57 MB -> 491.82 MB (17.8% reduction)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,...,prev__CNT_PAYMENT__max,prev__CNT_PAYMENT__min,prev__CNT_PAYMENT__sum,prev__CNT_PAYMENT__std,prev__DAYS_FIRST_DRAWING__count,prev__DAYS_FIRST_DRAWING__mean,prev__DAYS_FIRST_DRAWING__max,prev__DAYS_FIRST_DRAWING__min,prev__DAYS_FIRST_DRAWING__sum,prev__DAYS_FIRST_DRAWING__std,prev__DAYS_FIRST_DUE__count,prev__DAYS_FIRST_DUE__mean,prev__DAYS_FIRST_DUE__max,prev__DAYS_FIRST_DUE__min,prev__DAYS_FIRST_DUE__sum,prev__DAYS_FIRST_DUE__std,prev__DAYS_LAST_DUE_1ST_VERSION__count,prev__DAYS_LAST_DUE_1ST_VERSION__mean,prev__DAYS_LAST_DUE_1ST_VERSION__max,prev__DAYS_LAST_DUE_1ST_VERSION__min,prev__DAYS_LAST_DUE_1ST_VERSION__sum,prev__DAYS_LAST_DUE_1ST_VERSION__std,prev__DAYS_LAST_DUE__count,prev__DAYS_LAST_DUE__mean,prev__DAYS_LAST_DUE__max,prev__DAYS_LAST_DUE__min,prev__DAYS_LAST_DUE__sum,prev__DAYS_LAST_DUE__std,prev__DAYS_TERMINATION__count,prev__DAYS_TERMINATION__mean,prev__DAYS_TERMINATION__max,prev__DAYS_TERMINATION__min,prev__DAYS_TERMINATION__sum,prev__DAYS_TERMINATION__std,prev__NFLAG_INSURED_ON_APPROVAL__count,prev__NFLAG_INSURED_ON_APPROVAL__mean,prev__NFLAG_INSURED_ON_APPROVAL__max,prev__NFLAG_INSURED_ON_APPROVAL__min,prev__NFLAG_INSURED_ON_APPROVAL__sum,prev__NFLAG_INSURED_ON_APPROVAL__std,inst__NUM_INSTALMENT_VERSION__count,inst__NUM_INSTALMENT_VERSION__mean,inst__NUM_INSTALMENT_VERSION__max,inst__NUM_INSTALMENT_VERSION__min,inst__NUM_INSTALMENT_VERSION__sum,inst__NUM_INSTALMENT_VERSION__std,inst__DAYS_ENTRY_PAYMENT__count,inst__DAYS_ENTRY_PAYMENT__mean,inst__DAYS_ENTRY_PAYMENT__max,inst__DAYS_ENTRY_PAYMENT__min,inst__DAYS_ENTRY_PAYMENT__sum,inst__DAYS_ENTRY_PAYMENT__std,inst__AMT_INSTALMENT__count,inst__AMT_INSTALMENT__mean,inst__AMT_INSTALMENT__max,inst__AMT_INSTALMENT__min,inst__AMT_INSTALMENT__sum,inst__AMT_INSTALMENT__std,inst__AMT_PAYMENT__count,inst__AMT_PAYMENT__mean,inst__AMT_PAYMENT__max,inst__AMT_PAYMENT__min,inst__AMT_PAYMENT__sum,inst__AMT_PAYMENT__std,cc__AMT_BALANCE__count,cc__AMT_BALANCE__mean,cc__AMT_BALANCE__max,cc__AMT_BALANCE__min,cc__AMT_BALANCE__sum,cc__AMT_BALANCE__std,cc__AMT_CREDIT_LIMIT_ACTUAL__count,cc__AMT_CREDIT_LIMIT_ACTUAL__mean,cc__AMT_CREDIT_LIMIT_ACTUAL__max,cc__AMT_CREDIT_LIMIT_ACTUAL__min,cc__AMT_CREDIT_LIMIT_ACTUAL__sum,cc__AMT_CREDIT_LIMIT_ACTUA

In [31]:

# Cell 13: Missing values overview (top 30)

missing = df.isnull().mean().sort_values(ascending=False).head(30)
print(missing)

# Simple imputation strategy examples (do more in feature engineering notebook)

# For numeric aggregated columns, fill with -999 (Best for tree models like XGBoost)
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
agg_cols = [c for c in num_cols if '__' in c]  # aggregated columns have __ in names
for c in agg_cols:
    # Changed to assign back to df[c] to avoid FutureWarning and ensure inplace operation
    df[c] = df[c].fillna(-999) 

    
# For categorical columns in app, fillna with 'MISSING'
cat_cols = df.select_dtypes(include=['category','object']).columns.tolist()

for c in cat_cols:
    # ---  FIX START: Handling Categorical TypeError ---
    # 1. Add 'MISSING' to the column's category list first
    # This prevents the TypeError you were facing
    if 'MISSING' not in df[c].cat.categories:
        df[c] = df[c].cat.add_categories('MISSING')
        
    # 2. Now fill NA values with the newly added category
    df[c] = df[c].fillna('MISSING')
    # ---  FIX END ---

print("After simple imputation, NA% (should be 0 for many agg cols):", df.isnull().mean().sort_values(ascending=False).head(10))

COMMONAREA_MODE             0.698723
COMMONAREA_AVG              0.698723
COMMONAREA_MEDI             0.698723
NONLIVINGAPARTMENTS_MODE    0.694330
NONLIVINGAPARTMENTS_MEDI    0.694330
NONLIVINGAPARTMENTS_AVG     0.694330
FONDKAPREMONT_MODE          0.683862
LIVINGAPARTMENTS_MEDI       0.683550
LIVINGAPARTMENTS_AVG        0.683550
LIVINGAPARTMENTS_MODE       0.683550
FLOORSMIN_MODE              0.678486
FLOORSMIN_MEDI              0.678486
FLOORSMIN_AVG               0.678486
YEARS_BUILD_MODE            0.664978
YEARS_BUILD_MEDI            0.664978
YEARS_BUILD_AVG             0.664978
OWN_CAR_AGE                 0.659908
LANDAREA_AVG                0.593767
LANDAREA_MODE               0.593767
LANDAREA_MEDI               0.593767
BASEMENTAREA_MEDI           0.585160
BASEMENTAREA_AVG            0.585160
BASEMENTAREA_MODE           0.585160
EXT_SOURCE_1                0.563811
NONLIVINGAREA_AVG           0.551792
NONLIVINGAREA_MODE          0.551792
NONLIVINGAREA_MEDI          0.551792
E

In [32]:
# Cell 14: Save to parquet for next notebooks
out_path = os.path.join(OUT_DIR, "merged_application.parquet")
df.to_parquet(out_path, index=False)
print("Saved merged dataframe to:", out_path)
print("Merged shape:", df.shape)


Saved merged dataframe to: ../outputs\merged_application.parquet
Merged shape: (307511, 368)


In [33]:
# Cell 15: Quick checks
print("TARGET distribution (if present):")
if 'TARGET' in df.columns:
    print(df['TARGET'].value_counts(normalize=True) * 100)
print("Sample rows:")
display(df.sample(3))


TARGET distribution (if present):
TARGET
0    91.927118
1     8.072882
Name: proportion, dtype: float64
Sample rows:


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,...,prev__CNT_PAYMENT__max,prev__CNT_PAYMENT__min,prev__CNT_PAYMENT__sum,prev__CNT_PAYMENT__std,prev__DAYS_FIRST_DRAWING__count,prev__DAYS_FIRST_DRAWING__mean,prev__DAYS_FIRST_DRAWING__max,prev__DAYS_FIRST_DRAWING__min,prev__DAYS_FIRST_DRAWING__sum,prev__DAYS_FIRST_DRAWING__std,prev__DAYS_FIRST_DUE__count,prev__DAYS_FIRST_DUE__mean,prev__DAYS_FIRST_DUE__max,prev__DAYS_FIRST_DUE__min,prev__DAYS_FIRST_DUE__sum,prev__DAYS_FIRST_DUE__std,prev__DAYS_LAST_DUE_1ST_VERSION__count,prev__DAYS_LAST_DUE_1ST_VERSION__mean,prev__DAYS_LAST_DUE_1ST_VERSION__max,prev__DAYS_LAST_DUE_1ST_VERSION__min,prev__DAYS_LAST_DUE_1ST_VERSION__sum,prev__DAYS_LAST_DUE_1ST_VERSION__std,prev__DAYS_LAST_DUE__count,prev__DAYS_LAST_DUE__mean,prev__DAYS_LAST_DUE__max,prev__DAYS_LAST_DUE__min,prev__DAYS_LAST_DUE__sum,prev__DAYS_LAST_DUE__std,prev__DAYS_TERMINATION__count,prev__DAYS_TERMINATION__mean,prev__DAYS_TERMINATION__max,prev__DAYS_TERMINATION__min,prev__DAYS_TERMINATION__sum,prev__DAYS_TERMINATION__std,prev__NFLAG_INSURED_ON_APPROVAL__count,prev__NFLAG_INSURED_ON_APPROVAL__mean,prev__NFLAG_INSURED_ON_APPROVAL__max,prev__NFLAG_INSURED_ON_APPROVAL__min,prev__NFLAG_INSURED_ON_APPROVAL__sum,prev__NFLAG_INSURED_ON_APPROVAL__std,inst__NUM_INSTALMENT_VERSION__count,inst__NUM_INSTALMENT_VERSION__mean,inst__NUM_INSTALMENT_VERSION__max,inst__NUM_INSTALMENT_VERSION__min,inst__NUM_INSTALMENT_VERSION__sum,inst__NUM_INSTALMENT_VERSION__std,inst__DAYS_ENTRY_PAYMENT__count,inst__DAYS_ENTRY_PAYMENT__mean,inst__DAYS_ENTRY_PAYMENT__max,inst__DAYS_ENTRY_PAYMENT__min,inst__DAYS_ENTRY_PAYMENT__sum,inst__DAYS_ENTRY_PAYMENT__std,inst__AMT_INSTALMENT__count,inst__AMT_INSTALMENT__mean,inst__AMT_INSTALMENT__max,inst__AMT_INSTALMENT__min,inst__AMT_INSTALMENT__sum,inst__AMT_INSTALMENT__std,inst__AMT_PAYMENT__count,inst__AMT_PAYMENT__mean,inst__AMT_PAYMENT__max,inst__AMT_PAYMENT__min,inst__AMT_PAYMENT__sum,inst__AMT_PAYMENT__std,cc__AMT_BALANCE__count,cc__AMT_BALANCE__mean,cc__AMT_BALANCE__max,cc__AMT_BALANCE__min,cc__AMT_BALANCE__sum,cc__AMT_BALANCE__std,cc__AMT_CREDIT_LIMIT_ACTUAL__count,cc__AMT_CREDIT_LIMIT_ACTUAL__mean,cc__AMT_CREDIT_LIMIT_ACTUAL__max,cc__AMT_CREDIT_LIMIT_ACTUAL__min,cc__AMT_CREDIT_LIMIT_ACTUAL__sum,cc__AMT_CREDIT_LIMIT_ACTUA